# Hurst Exponent

The mean reversion trading strategy is based on the assumption that the price of the asset will revert back to its mean. Without making the assumption on stationarity, the speed of diffusion (i.e. How fast the price diffuses from its initial value) can be used to determine whether the asset price time series is suitable for mean reversion trading.

According to [Wikipedia](https://en.wikipedia.org/wiki/Hurst_exponent), the Hurst exponent is defined:
$$E[\frac{R(n)}{S(n)}] = Cn^H $$ 
$$\text{As} \space \text{n} -> \infty $$


$$\log (R/S)_{L_s} = \log C + H \log n$$

To estimate this Hurst exponent (Skipping mathematical notes, notes can be foudn in the Wikipedia):

For a subset time series of length n, $X = X_1, X_2, ..., X_n $, the rescaled range can be calculated as follows:
1. Calculate the mean
2. Create mean-adjusted series
3. Calcualte the cumulative deviate series Z
4. Compute the range R
5. Compute the standard deviation S

The Hurst exponent is estimated by fitting the linear line $\log (R/S)_{L_s}$ as a function of $\log n$

In my code repository, the Hurst exponent estimation function is constructed as below:

```python
def get_hurst_exponent_rs(price: npt.NDArray[np.float_]) -> npt.NDArray[np.float_]:
    """
    The function calculates the Hurst exponent using the R/S method for a given price
    series.

    Args:
      price (npt.NDArray[np.float_]): The parameter "price" is expected to be a
    1-dimensional numpy array of float values representing the price data.

    Returns:
      the Hurst exponent, which is calculated using the R/S method.
    """
    ret = calculate_daily_return(price)[1:]
    n = np.round(len(ret) / 2 ** np.arange(0, 10)).astype(int)

    y = []
    x = []

    for i in n:
        rs = []
        for t in range(0, round(len(ret) / i)):
            sub_ts = ret[t * i : (t + 1) * i]
            mean_adjusted = sub_ts - np.mean(sub_ts)
            cumul_mean_adjusted = np.cumsum(mean_adjusted)
            r = np.max(cumul_mean_adjusted) - np.min(cumul_mean_adjusted)
            s = np.std(sub_ts)
            rs.append(r / s)
        y.append(np.mean(rs))
        x.append(i)

    return np.polyfit(np.log(x), np.log(y), 1)[0]
```

In [2]:
import yfinance as yf

from py_quant.mean_reversion import hurst_exponent as he

if __name__ == "__main__":
    price = yf.download("CAD=X", start="2007-07-24", end="2012-03-28", progress=False)[
        "Adj Close"
    ].to_numpy()

    hurst_exponent1 = he.get_hurst_exponent_rs(price)
    print(f"Hurst Exponent with R/S analysis: {hurst_exponent1}")

Hurst Exponent with R/S analysis: 0.5751560049347784
